In [83]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(1, '../prediction/lstm/')
from LSTMpredict import LSTMpredict

import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import Counter
import os
from argparse import Namespace
import json
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import HBox, VBox

In [97]:
predict_obj = LSTMpredict(model_file_path='../data/model_lstm/', mc_model_file_path='../data/model_mc/', topk=5)
names_set = predict_obj.names_set[0]
reference_dict_val = predict_obj.reference_dict_val
reference_dict = predict_obj.reference_dict
inv_ref_dict = {val:key for key,val in reference_dict.items()}

In [98]:
style = {'description_width': 'initial', 'select_width' : 'initial'}
variableType = None

In [99]:
def display_archive_types(archive_types_list):
    archive_types_list.sort()
    archive_types_list.insert(0,'Select')
    return archive_types_list

In [100]:
archive_type_dropdown = widgets.Dropdown(options = display_archive_types(list(names_set)), description='Archive Type')

output_varType = widgets.Output()
output_proxy = widgets.Output()
output_units = widgets.Output()
output_int_var = widgets.Output()
output_int_var_det = widgets.Output()
output_inf_var = widgets.Output()
output_inf_var_units = widgets.Output()

var_type_dropdown = widgets.Dropdown(placeholder='Select',options = ['select','measured', 'inferred'], description='variable type'
        , style=style, ensure_option=True, disabled=False)
proxy_dropdown = widgets.Combobox(placeholder='Select',options = [], description='proxy observation type'
        , style=style, ensure_option=True, disabled=False)
units_dropdown = widgets.Combobox(placeholder='Select',options =[], description='units(Proxy Observation Type)'
        , style=style, ensure_option=True, disabled=False)
int_var_dropdown = widgets.Combobox(placeholder='Select',options = [], description='interpretation/variable'
        , style=style, ensure_option=True, disabled=False)
int_var_det_dropdown = widgets.Combobox(placeholder='Select',options = [], description='interpretation/variableDetail'
        , style=style, ensure_option=True, disabled=False)
inf_var_dropdown = widgets.Combobox(placeholder='Select',options = [], description='inferred variable'
        , style=style, ensure_option=True, disabled=False)
inf_var_units_dropdown = widgets.Combobox(placeholder='Select',options = [], description='units(Inferred Variable)'
        , style=style, ensure_option=True, disabled=False)

In [101]:
def inf_var_dropdown_eventhandler(change):
    global input_sent_list
    
    output_inf_var_units.clear_output()
    
    inf_var_type_val = change.new
    
    input_sent_list = input_sent_list[:4]
    input_sent_list.append(inf_var_type_val if inf_var_type_val in reference_dict_val else 'NA')
    
    isInf = True if variableType == 'inferred' else False
    results = predict_obj.predictForSentence(sentence=','.join(input_sent_list), isInferred=isInf)['0']
    results = [inv_ref_dict.get(val, val) for val in results]
    displayChildren(results, inf_var_units_dropdown, None, False, is_units=True)

    with output_inf_var_units:
        output_inf_var_units.clear_output()
        display(inf_var_units_dropdown)

In [102]:
def int_var_det_dropdown_eventhandler(change):
    global input_sent_list

    output_inf_var.clear_output()
    output_inf_var_units.clear_output()
    
    int_var_det_value = change.new
    
    input_sent_list = input_sent_list[:3]
    input_sent_list.append(int_var_det_value if int_var_det_value in reference_dict_val else 'NA')
    
    isInf = True if variableType == 'inferred' else False
    results = predict_obj.predictForSentence(sentence=','.join(input_sent_list), isInferred=isInf)['0']
    results = [inv_ref_dict.get(val, val) for val in results]
    displayChildren(results, inf_var_dropdown, inf_var_dropdown_eventhandler, True)

    with output_inf_var:
        output_inf_var.clear_output()
        display(inf_var_dropdown)

In [103]:
def int_var_dropdown_eventhandler(change):
    global input_sent_list
    
    output_int_var_det.clear_output()
    output_inf_var.clear_output()
    output_inf_var_units.clear_output()
    
    int_var_type_value = change.new
    
    input_sent_list = input_sent_list[:2]
    input_sent_list.append(int_var_type_value if int_var_type_value in reference_dict_val else 'NA')
    
    isInf = True if variableType == 'inferred' else False
    results = predict_obj.predictForSentence(sentence=','.join(input_sent_list), isInferred=isInf)['0']
    results = [inv_ref_dict.get(val, val) for val in results]
    displayChildren(results, int_var_det_dropdown, int_var_det_dropdown_eventhandler, True)

    with output_int_var_det:
        output_int_var_det.clear_output()
        display(int_var_det_dropdown)

In [104]:
def proxy_dropdown_eventhandler(change):
    global input_sent_list
    
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    output_inf_var.clear_output()
    output_inf_var_units.clear_output()
    
    proxy_obs_type_value = change.new
    
    input_sent_list = input_sent_list[:1]
    input_sent_list.append(proxy_obs_type_value if proxy_obs_type_value in reference_dict_val else 'NA')
    
    isInf = True if variableType == 'inferred' else False
    results_untis =  predict_obj.predictForSentence(sentence=','.join(input_sent_list), isInferred=isInf)['0']
    results = predict_obj.predictForSentence(sentence=','.join(input_sent_list), isInferred=isInf)['1']
    
    results_untis = [inv_ref_dict.get(val, val) for val in results_untis]
    results = [inv_ref_dict.get(val, val) for val in results]
    
    displayChildren(results_untis, units_dropdown, None, False, is_units=True)
    displayChildren(results, int_var_dropdown, int_var_dropdown_eventhandler, True)
    
    with output_units:
        output_units.clear_output()
        display(units_dropdown)
    with output_int_var:
        output_int_var.clear_output()
        display(int_var_dropdown)
    with output_int_var_det:
        output_int_var_det.clear_output()

In [105]:
def displayChildren(options_list, dropdown_obj, dropdown_eventhandler, to_observe, is_units = False):
    
    if is_units:
        if options_list and len(options_list) == 1 and options_list[0] == 'NA':
            options_list = ['Unitless']
        else:
            options_list = [x if x != 'NA' else 'Unitless' for x in options_list]   
    else:
        if options_list and len(options_list) == 1 and options_list[0] == 'NA':
            options_list = ['NA']

    dropdown_obj.options = options_list
    if to_observe:
        dropdown_obj.observe(dropdown_eventhandler, names='value')

In [106]:
def var_type_dropdown_eventhandler(change):
    global input_sent_list, variableType
    variableType = change.new
    
    output_proxy.clear_output()
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    output_inf_var.clear_output()
    output_inf_var_units.clear_output()
    
    isInf = True if variableType == 'inferred' else False

    if isInf:

        results = predict_obj.predictForSentence(sentence=','.join(input_sent_list[:1]), isInferred=isInf)['0']
        results = [inv_ref_dict.get(val, val) for val in results]
        displayChildren(results, inf_var_dropdown, inf_var_dropdown_eventhandler, True)

        with output_inf_var:
            output_inf_var.clear_output()
            display(inf_var_dropdown)
    else:

        results = predict_obj.predictForSentence(sentence=','.join(input_sent_list[:1]), isInferred=isInf)['0']
        results = [inv_ref_dict.get(val, val) for val in results]
        displayChildren(results, proxy_dropdown, proxy_dropdown_eventhandler, True)

        with output_proxy:
            output_proxy.clear_output()
            display(proxy_dropdown)

In [107]:
def archive_type_dropdown_eventhandler(change):
    global input_sent_list
    archive_type_value = change.new
    input_sent_list = []
    input_sent_list.append(archive_type_value if archive_type_value in reference_dict_val else 'NA')
    
    output_proxy.clear_output()
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    output_inf_var.clear_output()
    output_inf_var_units.clear_output()
    
    var_type_dropdown.observe(var_type_dropdown_eventhandler, names='value')
    with output_varType:
        output_varType.clear_output()
        display(var_type_dropdown)

In [108]:
archive_type_dropdown.observe(archive_type_dropdown_eventhandler, names='value')

display(archive_type_dropdown)

Dropdown(description='Archive Type', options=('Select', 'Coral', 'GlacierIce', 'Hybrid', 'LakeSediment', 'Mari…

In [109]:
zero_line = HBox([output_varType])
first_line = HBox([output_proxy])
second_line = HBox([output_units])
third_line = HBox([output_int_var, output_int_var_det])
fourth_line = HBox([output_inf_var, output_inf_var_units])
VBox([zero_line, first_line, second_line, third_line, fourth_line])